# Unsupervised Generative Modeling Using Matrix Product States

This notebook applies the MPS training method from the paper "Unsupervised Generative Modeling Using Matrix Product States" to the Bars and Stripes dataset.

<ins>Unsupervised Generative Modeling Using Matrix Product States </ins>\
Zhao-Yu Han, Jun Wang, Heng Fan, Lei Wang, and Pan Zhang \
Phys. Rev. X 8, 031012 – Published 17 July 2018 \
[https://doi.org/10.1103/PhysRevX.8.031012](https://doi.org/10.1103/PhysRevX.8.031012)

In [ ]:
%cd /workspaces/quantum-research
import numpy as np
import matplotlib.pyplot as plt

## Generate BAS Dataset

In [ ]:
def NumLinesStripes(a, b):
  return pow(2,a)+pow(2,b)-2


def LinesStripes(a, b, c):
  arr = np.zeros(a*b)
  c = c+1
  if (c<pow(2,a)): #lines
    lines = np.zeros (a, dtype = bool)
    for i in range (a):
      if (c%2 == 1):
        lines[i] = True
      c = int(c/2)
    index = 0
    for element in lines:
      if element:
        for i in range(index, index+b):
          arr[i] = 1
      index = index+b
  else: #stripes
    c = c-pow(2,a)
    stripes = np.zeros (b, dtype = bool)
    for i in range (b):
      if (c%2 == 1):
        stripes[i] = True
      c = int(c/2)
    index = 0
    for element in stripes:
      if element:
        for i in range(index, a*b, b):
          arr[i] = 1
      index = index+1
  return arr.astype(int)

def PrintLinesStripes(a, b, arr):
  index = 0
  for i in range(a):
    word = ''
    for j in range (b):
      if(arr[index] == 1):
        word = word + 'X'
      else:
        word = word + '0'
      index = index + 1
    print(word)

In [ ]:
dim = 6
num_samples = NumLinesStripes(dim, dim)

# generate bars and stripes samples
dataset = np.zeros((num_samples, dim**2), dtype=int)
for i in range(num_samples):
    sample = LinesStripes(dim, dim, i)
    dataset[i,:] = sample

print("Dataset Shape: ", dataset.shape)

## Run MPS on BAS dataset

In [ ]:
from mps.mps import MPS

m = MPS(dim**2)
m.left_cano()
m.designate_data(dataset)
m.init_cumulants()

m.cutoff = 5e-5
m.descent_step_length = 0.05
m.descent_steps = 10
m.train(10)

# Evaluate Loss

In [ ]:
# plot loss
loss = np.array(m.Loss)
plt.plot(np.arange(0,len(loss)), loss)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Generate Samples

In [ ]:
sample = m.generate_sample_1()
PrintLinesStripes(dim, dim, sample)